***GENERATED CODE FOR regressionmode PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'id', 'transformation_label': 'String Indexer'}], 'feature': 'id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AVpf3txeLJeJML43FN82', 'max': 'AVpf3txeLJeJML43FN82', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'id'}, {'feature_label': 'id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'asins', 'transformation_label': 'String Indexer'}], 'feature': 'asins', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B00IWQ3YPC', 'max': 'B01MUG578I,B072MP2CFG', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'asins'}, {'feature_label': 'asins', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('asins')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'brand', 'transformation_label': 'String Indexer'}], 'feature': 'brand', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Microsoft', 'max': 'Microsoft', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'brand'}, {'feature_label': 'brand', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('brand')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'categories', 'transformation_label': 'String Indexer'}], 'feature': 'categories', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Auto & Tires,Auto Electronics,Car Speakers and Subwoofers,10\\ Car Subwoofer,Car Subwoofers,All Car Subwoofers,Car Electronics & GPS,Car Audio,Car Subwoofers & Enclosures,Component Subwoofers,Consumer Electronics,Vehicle Electronics & GPS,eBay Motors,Parts & Accessories,Car Electronics,Subwoofers & Enclosures,Subwoofers,Electronics,Car & Vehicle Electronics',
                                                                                                                                                                                                                                                                      'max': 'stone products,electronics,Parts & Accessories,brick manufacturing,landmark stone,Car Electronics,brick manufacturing process,eBay Motors,natural stone,brick designs,Digital Media Receivers,Car Stereo Receivers,Vehicle Electronics & GPS,brick,Electronics Features,Video In-Dash Units w/o GPS,Car Video,thin brick,Car Audio In-Dash Units,Car Audio,Consumer Electronics,brick sizes,glen gery,manufactured stone,Car Video Units W/out GPS/Nav,Apple CarPlay Receivers,In-Dash with GPS,brick colors,Car Electronics & GPS', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'categories'}, {'feature_label': 'categories', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('categories')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'colors', 'transformation_label': 'String Indexer'}], 'feature': 'colors', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '', 'stddev': '', 'min': 'Black', 'max': 'Navy,Black,Gray,Blue', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'colors'}, {'feature_label': 'colors', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('colors')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'dateAdded', 'transformation_label': 'Extract Date'}], 'feature': 'dateAdded', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'dateAdded'}, {'feature_label': 'dateAdded', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('dateAdded')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'dateUpdated', 'transformation_label': 'Extract Date'}], 'feature': 'dateUpdated', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'dateUpdated'}, {'feature_label': 'dateUpdated', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('dateUpdated')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'dimension', 'transformation_label': 'String Indexer'}], 'feature': 'dimension', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '113', 'mean': '', 'stddev': '', 'min': '2.1 in x 6.5 in x 3.1 in', 'max': '9.1 in x 5.9 in x 2.8 in', 'missing': '175', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'dimension'}, {'feature_label': 'dimension', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('dimension')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ean', 'threshold': 2147483647.0, 'transformation_label': 'Binarizer'}], 'feature': 'ean', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '352'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'ean'}, {'feature_label': 'ean', 'threshold': 2147483647.0, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('ean')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'imageURLs', 'transformation_label': 'String Indexer'}], 'feature': 'imageURLs', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'https://i5.walmartimages.com/asr/2a41f6f0-844e-4ace-b7e3-63faef991173_1.2179afeb48142c34d2c84adf91b42b02.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/c4308cf0-7504-4e04-a058-c6813b71f355_1.3495db4167a1899dab6700fdc26873e4.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/e6891167-1e14-4065-a2ac-7caae27eff0e_1.6c44a72985e433683d3fd9e275aae51d.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/06a6725e-9f60-4c1e-8b9e-800f4e1b37b6_1.4d89206b740350ff08f68c302a6c3766.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/2bf30602-2d72-4322-a031-1f6b96490fdb_1.34227fb5d0665200f28d282f66cb1c44.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/61C2lYnfHNL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/41uj2RoSvVL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/511y4WjVhBL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/511ONk1ZiNL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/41GFiLfm8KL._SL1000_.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/4562/4562009_sa.jpg,https://i5.walmartimages.com/asr/505d2520-5276-49a3-b88f-979afb480d7a_1.6aed053b029ec373ce7030418ecafce2.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/1868aee8-d107-4b31-bb20-329063d5fdc6_1.c79adfe26d16a447b48984b8a06251ff.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/bd468836-1da2-4f3c-a754-38506807db49_1.0b428a6611b2af4aba06814ffa10fdcd.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/f81b84c9-c000-4a2c-a6a2-a80039c14b18_1.502d1ce0283b5a6c0508c95f5d480bc4.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/41X1El9m6wL.jpg,https://www.barcodable.com/images/barcode/0889842011050.png,https://static.bhphoto.com/images/itemImgPlaceholder.jpg,https://static.bhphoto.com/images/images500x500/microsoft_rh7_00001_surface_pro_4_type_1445016092000_1188746.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/4562/4562009_sa.jpg,http://images.frys.com/art/product/box_shots/8639000.box.GIF',
                                                                                                                                                                                                                                                                    'max': 'https://i5.walmartimages.com/asr/2a41f6f0-844e-4ace-b7e3-63faef991173_1.2179afeb48142c34d2c84adf91b42b02.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/c4308cf0-7504-4e04-a058-c6813b71f355_1.3495db4167a1899dab6700fdc26873e4.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/e6891167-1e14-4065-a2ac-7caae27eff0e_1.6c44a72985e433683d3fd9e275aae51d.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/06a6725e-9f60-4c1e-8b9e-800f4e1b37b6_1.4d89206b740350ff08f68c302a6c3766.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/2bf30602-2d72-4322-a031-1f6b96490fdb_1.34227fb5d0665200f28d282f66cb1c44.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/61C2lYnfHNL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/41uj2RoSvVL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/511y4WjVhBL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/511ONk1ZiNL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/41GFiLfm8KL._SL1000_.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/4562/4562009_sa.jpg,https://i5.walmartimages.com/asr/505d2520-5276-49a3-b88f-979afb480d7a_1.6aed053b029ec373ce7030418ecafce2.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/1868aee8-d107-4b31-bb20-329063d5fdc6_1.c79adfe26d16a447b48984b8a06251ff.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/bd468836-1da2-4f3c-a754-38506807db49_1.0b428a6611b2af4aba06814ffa10fdcd.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/f81b84c9-c000-4a2c-a6a2-a80039c14b18_1.502d1ce0283b5a6c0508c95f5d480bc4.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/41X1El9m6wL.jpg,https://www.barcodable.com/images/barcode/0889842011050.png,https://static.bhphoto.com/images/itemImgPlaceholder.jpg,https://static.bhphoto.com/images/images500x500/microsoft_rh7_00001_surface_pro_4_type_1445016092000_1188746.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/4562/4562009_sa.jpg,http://images.frys.com/art/product/box_shots/8639000.box.GIF', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'imageURLs'}, {'feature_label': 'imageURLs', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('imageURLs')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'keys', 'transformation_label': 'String Indexer'}], 'feature': 'keys', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '047532907711,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/4525009,ihome/ibt33bc,sditechnologies/ibt33bc,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/555013051,ihomerechargeablesplashproofstereobluetoothspeakerblackibt33bc/b00wbs31oq,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/b00wbs31oq',
                                                                                                                                                                                                                                                          'max': 'verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/b0172ds3me,novatel/mifi6620l,verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/9940096,649496019587,verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/b00ntjkaxg,0649496019587,verizonwireless/mifi6620l', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'keys'}, {'feature_label': 'keys', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('keys')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'manufacturer', 'transformation_label': 'String Indexer'}], 'feature': 'manufacturer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '261', 'mean': '', 'stddev': '', 'min': 'Lenovo Idea', 'max': 'iHome', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'manufacturer'}, {'feature_label': 'manufacturer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('manufacturer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'manufacturerNumber', 'transformation_label': 'String Indexer'}], 'feature': 'manufacturerNumber', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '45W SLIM AC ADAPTER - 88801419', 'max': 'Mifi6620L', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'manufacturerNumber'}, {'feature_label': 'manufacturerNumber', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('manufacturerNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'name', 'transformation_label': 'String Indexer'}], 'feature': 'name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Microsoft Surface Pro 4 Type Cover with Fingerprint ID', 'max': 'Microsoft Surface Pro 4 Type Cover with Fingerprint ID', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'name'}, {'feature_label': 'name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'primaryCategories', 'transformation_label': 'String Indexer'}], 'feature': 'primaryCategories', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Electronics', 'max': 'Electronics', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'primaryCategories'}, {'feature_label': 'primaryCategories', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('primaryCategories')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewsdate', 'transformation_label': 'Extract Date'}], 'feature': 'reviewsdate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '6'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'reviewsdate'}, {'feature_label': 'reviewsdate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('reviewsdate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewsdateSeen', 'transformation_label': 'String Indexer'}], 'feature': 'reviewsdateSeen', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '2014-10-24T00:00:00Z', 'max': '2018-04-28T00:00:00Z', 'missing': '0', 'distinct': '75'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewsdateSeen'}, {'feature_label': 'reviewsdateSeen', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewsdateSeen')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewssourceURLs', 'transformation_label': 'String Indexer'}], 'feature': 'reviewssourceURLs', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'http://reviews.bestbuy.com/3545/2877396/reviews.htm?format=embedded,https://reviews.bestbuy.com/3545/2877396/reviews.htm?format=embedded', 'max': 'https://www.walmart.com/reviews/product/51933274', 'missing': '0', 'distinct': '201'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewssourceURLs'}, {'feature_label': 'reviewssourceURLs', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewssourceURLs')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewstext', 'transformation_label': 'String Indexer'}], 'feature': 'reviewstext', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '"Due to my position, I have to have security on my Electronics that often contain Protected Health Information (PHI).I had this in mind when I purchased the biometric keyboard for my Microsoft Surface Pro 4.So the good news is that the biometric meter does work....most of the time!At times, the biometric device shows an ""error"" unable to read message that it credits', 'max': 'works great, was perfect for a meeting and taking the minutes.', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewstext'}, {'feature_label': 'reviewstext', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewstext')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewstitle', 'transformation_label': 'String Indexer'}], 'feature': 'reviewstitle', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' for an extra $10 the added benefit of a fingerprint reader can\'t be beat."', 'max': 'works perfectly', 'missing': '0', 'distinct': '465'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewstitle'}, {'feature_label': 'reviewstitle', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewstitle')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewsusername', 'transformation_label': 'String Indexer'}], 'feature': 'reviewsusername', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '5124.0', 'stddev': 'nan', 'min': ' but I really like the lights on this speaker. There is the outer ring that surrounds the speaker on the front that changes different colors and two white LED ""strobe"" lights on either side. You can set the lights to dance with the music', 'max': 'zombiepie69-7', 'missing': '0', 'distinct': '483'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewsusername'}, {'feature_label': 'reviewsusername', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewsusername')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sourceURLs', 'transformation_label': 'String Indexer'}], 'feature': 'sourceURLs', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': ' I am very pleased with my purchase."',
                                                                                                                                                                                                                                                                      'max': 'https://www.walmart.com/ip/The-House-of-Marley-Rebel-Bluetooth-On-Ear-Headphones-with-Microphone/51933274,https://reviews.bestbuy.com/3545/5688025/reviews.htm?format=embedded,https://www.bestbuy.com/site/products/5688025.p,http://www.bestbuy.com/site/products/5688025.p,https://www.bhphotovideo.com/c/product/1283417-REG/house_of_marley_em_jh101_bk_rebel_bt_on_ear_bluetooth.html,http://www.bestbuy.com/site/the-house-of-marley-rebel-bt-wireless-on-ear-headphones-black/5688025.p?skuId=5688025,https://www.walmart.com/reviews/product/51933274,https://www.bestbuy.com/site/the-house-of-marley-rebel-bt-wireless-on-ear-headphones-black/5688025.p?skuId=5688025,https://www.amazon.com/House-Marley-Rebel-Bluetooth-Headphones/dp/B01700RNIO/,https://www.amazon.com/House-Marley-EM-JH101-BK-Bluetooth-Headphones/dp/B01700RNIO/', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sourceURLs'}, {'feature_label': 'sourceURLs', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sourceURLs')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'upc', 'transformation_label': 'String Indexer'}], 'feature': 'upc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '502371306583.27', 'stddev': '390158586199.42', 'min': ' I hope I get similar longevity from this one. We\'ll see..."', 'max': 'https://www.upccodesearch.com/upc/793276012944,http://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?id=1219405821712&skuId=9254006#tabbed-customerreviews,https://www.glengery.com/brick-products/view-brick-products/item/164982757-alpine,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded,http://www.ebay.com/itm/Defective-Alpine-iLX-007-7-Digital-Media-Receiver-In-Dash-Receiver-1533sw-/302431154490?hash=item466a4d313a,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=11,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=10,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=13,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=12,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=14,https://www.barcodable.com/upc/793276012944,http://www.amazon.com/Alpine-ILX-007/dp/B00Q397RJY,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=9,http://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?skuId=9254006,http://www.ebay.com/itm/Alpine-iLX-007-In-Dash-Receiver-/121935848708?hash=item1c63f16504:g:yk4AAOSwxcRW9Z16,https://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?skuId=9254006,http://www.amazon.com/Alpine-SWR-M100-Inch-Marine-Subwoofer/dp/B005LKCDR2,http://www.amazon.com/Alpine-Type-S-SPS-600C-speaker-component/dp/B001447Z8Q,http://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?id=1219405821712&skuId=9254006,http://www.amazon.com/Alpine-iLX-007-ILX-007/dp/B00Q397RJY,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=2,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=3,http://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?id=1219405821712&skuId=9254006#tab=buyingOptions?bof=openbox,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=4,http://www.ebay.com/itm/Alpine-ILX-107-2-DIN-Digital-Media-Rcvr-5-4GHz-Wireless-Apple-Car-Play-iPhone/292217711461?hash=item4409886b65%3Ag%3AClQAAOSwlTNZlQAb,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=5,http://www.ebay.com/itm/Alpine-iLX-007-7-Digital-Media-Receiver-In-Dash-With-Apple-CarPlay-1448sw-/282611072502?hash=item41cceebdf6,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=6,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=7,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=8', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'upc'}, {'feature_label': 'upc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('upc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'weight', 'transformation_label': 'String Indexer'}], 'feature': 'weight', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '890000000000.0', 'stddev': '0.0', 'min': '1.1 pounds', 'max': 'https://www.walmart.com/ip/Microsoft-Surface-Pro-4-Type-Cover-with-Fingerprint-ID/47167973,https://www.amazon.com/Microsoft-Surface-Cover-Fingerprint-Black/dp/B0168YIWSI/,https://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?skuId=4562009,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=51,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=43,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=53,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=52,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=44,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=49,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=46,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=45,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=50,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=47,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=48,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=33,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=42,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=40,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=39,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=41,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=38,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=31,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=37,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=36,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=32,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=35,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=34,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=30,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=25,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=26,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=29,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=28,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=22,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=24,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=23,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=21,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=27,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=20,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=19,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=16,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=15,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=17,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=9,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=18,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=11,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=12,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=13,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=10,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=14,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=6,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=3,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=8,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=4,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=5,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=7,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=2,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded,https://www.barcodable.com/upc/889842011050,https://www.bhphotovideo.com/c/product/1188746-REG/microsoft_rh7_00001_surface_pro_4_type.html,https://www.walmart.com/ip/Microsoft-Surface-Pro-4-Type-Cover-with-Fingerprint-ID-Black/47167973,https://www.walmart.com/reviews/product/47167973?limit=20&sort=relevancy&page=4,https://www.walmart.com/reviews/product/47167973,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?skuId=4562009,https://www.walmart.com/reviews/product/47167973?limit=20&sort=relevancy&page=2,https://www.amazon.com/Microsoft-Surface-Cover-Fingerprint-Black/dp/B01E3Q2H6G,https://www.walmart.com/reviews/product/47167973?limit=20&sort=relevancy&page=3,http://frys.com/product/8639000,http://frys.com/product/8639000?site=sr:SEARCH:MAIN_RSLT_PG,https://www.amazon.com/Microsoft-Surface-Cover-Fingerprint-Black/dp/B0168YIWSI,http://www.amazon.com/product-reviews/B0168YIWSI/,http://www.amazon.com/Microsoft-Surface-Cover-Fingerprint-Black/dp/B0168YIWSI,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?id=bb4562009&skuId=4562009#tabbed-customerreviews,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?id=bb4562009&skuId=4562009,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?id=bb4562009&skuId=4562009#tab=buyingOptions?bof=openbox,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?id=bb4562009&skuId=4562009&reviewsPage=2,http://www.bestbuy.com/site/olspage.jsp;jsessionid=DB10B7324359B034F2805C27203FF931.bbolsp-app04-172?skuId=4562009&type=product&id=bb4562009', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'weight'}, {'feature_label': 'weight', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('weight')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run regressionmodeHooks.ipynb
try:
	#sourcePreExecutionHook()

	electronicsproductdatanewupdate = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/ElectronicsProductDataNewUpdate.csv', 'filename': 'ElectronicsProductDataNewUpdate.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(electronicsproductdatanewupdate)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regressionmodeHooks.ipynb
try:
	#transformationPreExecutionHook()

	regressionmodeautofe = TransformationMain.run(electronicsproductdatanewupdate,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "id", "transformation_label": "String Indexer"}], "feature": "id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AVpf3txeLJeJML43FN82", "max": "AVpf3txeLJeJML43FN82", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "id"}, {"transformationsData": [{"feature_label": "asins", "transformation_label": "String Indexer"}], "feature": "asins", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B00IWQ3YPC", "max": "B01MUG578I,B072MP2CFG", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "asins"}, {"transformationsData": [{"feature_label": "brand", "transformation_label": "String Indexer"}], "feature": "brand", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Microsoft", "max": "Microsoft", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "brand"}, {"transformationsData": [{"feature_label": "categories", "transformation_label": "String Indexer"}], "feature": "categories", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Auto & Tires,Auto Electronics,Car Speakers and Subwoofers,10\\ Car Subwoofer,Car Subwoofers,All Car Subwoofers,Car Electronics & GPS,Car Audio,Car Subwoofers & Enclosures,Component Subwoofers,Consumer Electronics,Vehicle Electronics & GPS,eBay Motors,Parts & Accessories,Car Electronics,Subwoofers & Enclosures,Subwoofers,Electronics,Car & Vehicle Electronics", "max": "stone products,electronics,Parts & Accessories,brick manufacturing,landmark stone,Car Electronics,brick manufacturing process,eBay Motors,natural stone,brick designs,Digital Media Receivers,Car Stereo Receivers,Vehicle Electronics & GPS,brick,Electronics Features,Video In-Dash Units w/o GPS,Car Video,thin brick,Car Audio In-Dash Units,Car Audio,Consumer Electronics,brick sizes,glen gery,manufactured stone,Car Video Units W/out GPS/Nav,Apple CarPlay Receivers,In-Dash with GPS,brick colors,Car Electronics & GPS", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "categories"}, {"transformationsData": [{"feature_label": "colors", "transformation_label": "String Indexer"}], "feature": "colors", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "", "stddev": "", "min": "Black", "max": "Navy,Black,Gray,Blue", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "colors"}, {"transformationsData": [{"feature_label": "dateAdded", "transformation_label": "Extract Date"}], "feature": "dateAdded", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "dateAdded"}, {"transformationsData": [{"feature_label": "dateUpdated", "transformation_label": "Extract Date"}], "feature": "dateUpdated", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "dateUpdated"}, {"transformationsData": [{"feature_label": "dimension", "transformation_label": "String Indexer"}], "feature": "dimension", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "113", "mean": "", "stddev": "", "min": "2.1 in x 6.5 in x 3.1 in", "max": "9.1 in x 5.9 in x 2.8 in", "missing": "175", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "dimension"}, {"transformationsData": [{"feature_label": "ean", "threshold": 2147483647.0, "transformation_label": "Binarizer"}], "feature": "ean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "352"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "ean"}, {"transformationsData": [{"feature_label": "imageURLs", "transformation_label": "String Indexer"}], "feature": "imageURLs", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "https://i5.walmartimages.com/asr/2a41f6f0-844e-4ace-b7e3-63faef991173_1.2179afeb48142c34d2c84adf91b42b02.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/c4308cf0-7504-4e04-a058-c6813b71f355_1.3495db4167a1899dab6700fdc26873e4.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/e6891167-1e14-4065-a2ac-7caae27eff0e_1.6c44a72985e433683d3fd9e275aae51d.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/06a6725e-9f60-4c1e-8b9e-800f4e1b37b6_1.4d89206b740350ff08f68c302a6c3766.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/2bf30602-2d72-4322-a031-1f6b96490fdb_1.34227fb5d0665200f28d282f66cb1c44.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/61C2lYnfHNL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/41uj2RoSvVL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/511y4WjVhBL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/511ONk1ZiNL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/41GFiLfm8KL._SL1000_.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/4562/4562009_sa.jpg,https://i5.walmartimages.com/asr/505d2520-5276-49a3-b88f-979afb480d7a_1.6aed053b029ec373ce7030418ecafce2.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/1868aee8-d107-4b31-bb20-329063d5fdc6_1.c79adfe26d16a447b48984b8a06251ff.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/bd468836-1da2-4f3c-a754-38506807db49_1.0b428a6611b2af4aba06814ffa10fdcd.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/f81b84c9-c000-4a2c-a6a2-a80039c14b18_1.502d1ce0283b5a6c0508c95f5d480bc4.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/41X1El9m6wL.jpg,https://www.barcodable.com/images/barcode/0889842011050.png,https://static.bhphoto.com/images/itemImgPlaceholder.jpg,https://static.bhphoto.com/images/images500x500/microsoft_rh7_00001_surface_pro_4_type_1445016092000_1188746.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/4562/4562009_sa.jpg,http://images.frys.com/art/product/box_shots/8639000.box.GIF", "max": "https://i5.walmartimages.com/asr/2a41f6f0-844e-4ace-b7e3-63faef991173_1.2179afeb48142c34d2c84adf91b42b02.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/c4308cf0-7504-4e04-a058-c6813b71f355_1.3495db4167a1899dab6700fdc26873e4.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/e6891167-1e14-4065-a2ac-7caae27eff0e_1.6c44a72985e433683d3fd9e275aae51d.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/06a6725e-9f60-4c1e-8b9e-800f4e1b37b6_1.4d89206b740350ff08f68c302a6c3766.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/2bf30602-2d72-4322-a031-1f6b96490fdb_1.34227fb5d0665200f28d282f66cb1c44.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/61C2lYnfHNL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/41uj2RoSvVL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/511y4WjVhBL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/511ONk1ZiNL._SL1000_.jpg,https://images-na.ssl-images-amazon.com/images/I/41GFiLfm8KL._SL1000_.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/4562/4562009_sa.jpg,https://i5.walmartimages.com/asr/505d2520-5276-49a3-b88f-979afb480d7a_1.6aed053b029ec373ce7030418ecafce2.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/1868aee8-d107-4b31-bb20-329063d5fdc6_1.c79adfe26d16a447b48984b8a06251ff.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/bd468836-1da2-4f3c-a754-38506807db49_1.0b428a6611b2af4aba06814ffa10fdcd.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/f81b84c9-c000-4a2c-a6a2-a80039c14b18_1.502d1ce0283b5a6c0508c95f5d480bc4.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/41X1El9m6wL.jpg,https://www.barcodable.com/images/barcode/0889842011050.png,https://static.bhphoto.com/images/itemImgPlaceholder.jpg,https://static.bhphoto.com/images/images500x500/microsoft_rh7_00001_surface_pro_4_type_1445016092000_1188746.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/4562/4562009_sa.jpg,http://images.frys.com/art/product/box_shots/8639000.box.GIF", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "imageURLs"}, {"transformationsData": [{"feature_label": "keys", "transformation_label": "String Indexer"}], "feature": "keys", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "047532907711,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/4525009,ihome/ibt33bc,sditechnologies/ibt33bc,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/555013051,ihomerechargeablesplashproofstereobluetoothspeakerblackibt33bc/b00wbs31oq,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/b00wbs31oq", "max": "verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/b0172ds3me,novatel/mifi6620l,verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/9940096,649496019587,verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/b00ntjkaxg,0649496019587,verizonwireless/mifi6620l", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "keys"}, {"transformationsData": [{"feature_label": "manufacturer", "transformation_label": "String Indexer"}], "feature": "manufacturer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "261", "mean": "", "stddev": "", "min": "Lenovo Idea", "max": "iHome", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "manufacturer"}, {"transformationsData": [{"feature_label": "manufacturerNumber", "transformation_label": "String Indexer"}], "feature": "manufacturerNumber", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "45W SLIM AC ADAPTER - 88801419", "max": "Mifi6620L", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "manufacturerNumber"}, {"transformationsData": [{"feature_label": "name", "transformation_label": "String Indexer"}], "feature": "name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Microsoft Surface Pro 4 Type Cover with Fingerprint ID", "max": "Microsoft Surface Pro 4 Type Cover with Fingerprint ID", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "name"}, {"transformationsData": [{"feature_label": "primaryCategories", "transformation_label": "String Indexer"}], "feature": "primaryCategories", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Electronics", "max": "Electronics", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "primaryCategories"}, {"transformationsData": [{"feature_label": "reviewsdate", "transformation_label": "Extract Date"}], "feature": "reviewsdate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "6"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "reviewsdate"}, {"transformationsData": [{"feature_label": "reviewsdateSeen", "transformation_label": "String Indexer"}], "feature": "reviewsdateSeen", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "2014-10-24T00:00:00Z", "max": "2018-04-28T00:00:00Z", "missing": "0", "distinct": "75"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewsdateSeen"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsdoRecommend", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "441", "mean": "0.95", "stddev": "0.22", "min": "0", "max": "1", "missing": "32"}, "updatedLabel": "reviewsdoRecommend"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsnumHelpful", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "429", "mean": "0.61", "stddev": "2.97", "min": "0", "max": "47", "missing": "32"}, "updatedLabel": "reviewsnumHelpful"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsrating", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "495", "mean": "4.61", "stddev": "0.77", "min": "1", "max": "5", "missing": "20"}, "updatedLabel": "reviewsrating"}, {"transformationsData": [{"feature_label": "reviewssourceURLs", "transformation_label": "String Indexer"}], "feature": "reviewssourceURLs", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "http://reviews.bestbuy.com/3545/2877396/reviews.htm?format=embedded,https://reviews.bestbuy.com/3545/2877396/reviews.htm?format=embedded", "max": "https://www.walmart.com/reviews/product/51933274", "missing": "0", "distinct": "201"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewssourceURLs"}, {"transformationsData": [{"feature_label": "reviewstext", "transformation_label": "String Indexer"}], "feature": "reviewstext", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "\"Due to my position, I have to have security on my Electronics that often contain Protected Health Information (PHI).I had this in mind when I purchased the biometric keyboard for my Microsoft Surface Pro 4.So the good news is that the biometric meter does work....most of the time!At times, the biometric device shows an \"\"error\"\" unable to read message that it credits", "max": "works great, was perfect for a meeting and taking the minutes.", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewstext"}, {"transformationsData": [{"feature_label": "reviewstitle", "transformation_label": "String Indexer"}], "feature": "reviewstitle", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " for an extra $10 the added benefit of a fingerprint reader can't be beat.\"", "max": "works perfectly", "missing": "0", "distinct": "465"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewstitle"}, {"transformationsData": [{"feature_label": "reviewsusername", "transformation_label": "String Indexer"}], "feature": "reviewsusername", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "5124.0", "stddev": "nan", "min": " but I really like the lights on this speaker. There is the outer ring that surrounds the speaker on the front that changes different colors and two white LED \"\"strobe\"\" lights on either side. You can set the lights to dance with the music", "max": "zombiepie69-7", "missing": "0", "distinct": "483"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewsusername"}, {"transformationsData": [{"feature_label": "sourceURLs", "transformation_label": "String Indexer"}], "feature": "sourceURLs", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " I am very pleased with my purchase.\"", "max": "https://www.walmart.com/ip/The-House-of-Marley-Rebel-Bluetooth-On-Ear-Headphones-with-Microphone/51933274,https://reviews.bestbuy.com/3545/5688025/reviews.htm?format=embedded,https://www.bestbuy.com/site/products/5688025.p,http://www.bestbuy.com/site/products/5688025.p,https://www.bhphotovideo.com/c/product/1283417-REG/house_of_marley_em_jh101_bk_rebel_bt_on_ear_bluetooth.html,http://www.bestbuy.com/site/the-house-of-marley-rebel-bt-wireless-on-ear-headphones-black/5688025.p?skuId=5688025,https://www.walmart.com/reviews/product/51933274,https://www.bestbuy.com/site/the-house-of-marley-rebel-bt-wireless-on-ear-headphones-black/5688025.p?skuId=5688025,https://www.amazon.com/House-Marley-Rebel-Bluetooth-Headphones/dp/B01700RNIO/,https://www.amazon.com/House-Marley-EM-JH101-BK-Bluetooth-Headphones/dp/B01700RNIO/", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sourceURLs"}, {"transformationsData": [{"feature_label": "upc", "transformation_label": "String Indexer"}], "feature": "upc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "502371306583.27", "stddev": "390158586199.42", "min": " I hope I get similar longevity from this one. We'll see...\"", "max": "https://www.upccodesearch.com/upc/793276012944,http://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?id=1219405821712&skuId=9254006#tabbed-customerreviews,https://www.glengery.com/brick-products/view-brick-products/item/164982757-alpine,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded,http://www.ebay.com/itm/Defective-Alpine-iLX-007-7-Digital-Media-Receiver-In-Dash-Receiver-1533sw-/302431154490?hash=item466a4d313a,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=11,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=10,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=13,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=12,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=14,https://www.barcodable.com/upc/793276012944,http://www.amazon.com/Alpine-ILX-007/dp/B00Q397RJY,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=9,http://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?skuId=9254006,http://www.ebay.com/itm/Alpine-iLX-007-In-Dash-Receiver-/121935848708?hash=item1c63f16504:g:yk4AAOSwxcRW9Z16,https://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?skuId=9254006,http://www.amazon.com/Alpine-SWR-M100-Inch-Marine-Subwoofer/dp/B005LKCDR2,http://www.amazon.com/Alpine-Type-S-SPS-600C-speaker-component/dp/B001447Z8Q,http://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?id=1219405821712&skuId=9254006,http://www.amazon.com/Alpine-iLX-007-ILX-007/dp/B00Q397RJY,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=2,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=3,http://www.bestbuy.com/site/alpine-7-digital-media-receiver-apple-carplay-in-dash-receiver-black/9254006.p?id=1219405821712&skuId=9254006#tab=buyingOptions?bof=openbox,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=4,http://www.ebay.com/itm/Alpine-ILX-107-2-DIN-Digital-Media-Rcvr-5-4GHz-Wireless-Apple-Car-Play-iPhone/292217711461?hash=item4409886b65%3Ag%3AClQAAOSwlTNZlQAb,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=5,http://www.ebay.com/itm/Alpine-iLX-007-7-Digital-Media-Receiver-In-Dash-With-Apple-CarPlay-1448sw-/282611072502?hash=item41cceebdf6,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=6,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=7,http://reviews.bestbuy.com/3545/9254006/reviews.htm?format=embedded&page=8", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "upc"}, {"transformationsData": [{"feature_label": "weight", "transformation_label": "String Indexer"}], "feature": "weight", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "890000000000.0", "stddev": "0.0", "min": "1.1 pounds", "max": "https://www.walmart.com/ip/Microsoft-Surface-Pro-4-Type-Cover-with-Fingerprint-ID/47167973,https://www.amazon.com/Microsoft-Surface-Cover-Fingerprint-Black/dp/B0168YIWSI/,https://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?skuId=4562009,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=51,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=43,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=53,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=52,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=44,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=49,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=46,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=45,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=50,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=47,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=48,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=33,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=42,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=40,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=39,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=41,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=38,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=31,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=37,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=36,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=32,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=35,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=34,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=30,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=25,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=26,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=29,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=28,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=22,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=24,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=23,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=21,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=27,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=20,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=19,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=16,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=15,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=17,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=9,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=18,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=11,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=12,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=13,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=10,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=14,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=6,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=3,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=8,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=4,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=5,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=7,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded&page=2,http://reviews.bestbuy.com/3545/4562009/reviews.htm?format=embedded,https://www.barcodable.com/upc/889842011050,https://www.bhphotovideo.com/c/product/1188746-REG/microsoft_rh7_00001_surface_pro_4_type.html,https://www.walmart.com/ip/Microsoft-Surface-Pro-4-Type-Cover-with-Fingerprint-ID-Black/47167973,https://www.walmart.com/reviews/product/47167973?limit=20&sort=relevancy&page=4,https://www.walmart.com/reviews/product/47167973,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?skuId=4562009,https://www.walmart.com/reviews/product/47167973?limit=20&sort=relevancy&page=2,https://www.amazon.com/Microsoft-Surface-Cover-Fingerprint-Black/dp/B01E3Q2H6G,https://www.walmart.com/reviews/product/47167973?limit=20&sort=relevancy&page=3,http://frys.com/product/8639000,http://frys.com/product/8639000?site=sr:SEARCH:MAIN_RSLT_PG,https://www.amazon.com/Microsoft-Surface-Cover-Fingerprint-Black/dp/B0168YIWSI,http://www.amazon.com/product-reviews/B0168YIWSI/,http://www.amazon.com/Microsoft-Surface-Cover-Fingerprint-Black/dp/B0168YIWSI,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?id=bb4562009&skuId=4562009#tabbed-customerreviews,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?id=bb4562009&skuId=4562009,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?id=bb4562009&skuId=4562009#tab=buyingOptions?bof=openbox,http://www.bestbuy.com/site/microsoft-surface-pro-type-cover-with-fingerprint-id-black/4562009.p?id=bb4562009&skuId=4562009&reviewsPage=2,http://www.bestbuy.com/site/olspage.jsp;jsessionid=DB10B7324359B034F2805C27203FF931.bbolsp-app04-172?skuId=4562009&type=product&id=bb4562009", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "weight"}]}))

	#transformationPostExecutionHook(regressionmodeautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regressionmodeHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(regressionmodeautofe, ["reviewsdoRecommend", "reviewsnumHelpful", "id_stringindexer", "asins_stringindexer", "brand_stringindexer", "categories_stringindexer", "colors_stringindexer", "dateAdded_dayofmonth", "dateAdded_month", "dateAdded_year", "dateUpdated_dayofmonth", "dateUpdated_month", "dateUpdated_year", "dimension_stringindexer", "ean_binarizer", "imageURLs_stringindexer", "keys_stringindexer", "manufacturer_stringindexer", "manufacturerNumber_stringindexer", "name_stringindexer", "primaryCategories_stringindexer", "reviewsdate_dayofmonth", "reviewsdate_month", "reviewsdate_year", "reviewsdateSeen_stringindexer", "reviewssourceURLs_stringindexer", "reviewstext_stringindexer", "reviewstitle_stringindexer", "reviewsusername_stringindexer", "sourceURLs_stringindexer", "upc_stringindexer", "weight_stringindexer"], "reviewsrating")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

